### Step 3
# Run completions on the Q/A data set

# Setup
```bash
git clone https://github.com/Rational-Exponent/dispatcher.git lib/dispatcher
cd lib/dispatcher
pip install -e .
```


In [ ]:
import sys, os
lib_dir = os.path.abspath(os.path.join(os.path.curdir, "lib/dispatcher/src"))
if lib_dir not in sys.path:
    print(f"Adding {lib_dir} to sys.path")
    sys.path.insert(0, lib_dir)
else:
    print(f"{lib_dir} already in sys.path")

os.environ["APP_CONFIG_DIR"] = os.path.abspath(os.path.join(os.path.curdir, "lib/dispatcher/config"))

data_path = os.path.abspath(os.path.join(os.path.curdir, "..", "..", "eval-data"))


# Load Q/A set

In [ ]:
import json

qa_source_file = 'sample-occ-uce-mc-exam.json'

with open(os.path.join(data_path, qa_source_file), 'r') as f:
    file_data = f.read()
    qa_data = json.loads(file_data)

print(f"Loaded {len(qa_data.get("eval-questions"))} questions from {qa_source_file}")

# Run completions


In [ ]:
from fastapi.testclient import TestClient
from httpx import AsyncClient

# Import our application code to test
# We will test the top-level API application here
from dispatcher.main import api
from dispatcher.models import (
    CreateChatCompletionRequest,
    ChatCompletionRequestUserMessage
)

# Attach a test client to our imported FastAPI application
client = TestClient(api) 
# client = AsyncClient(app=api, base_url="http://test") 
 
for qa in qa_data.get("eval-questions"):
    request_payload = {
        "model": "passthrough", 
        "messages": [
            {
                "role": "user",
                "content": qa.get("question")
            }
        ],
        "stream": "false" 
    }

    # We need to patch the `validate_jwt` function to avoid authentication failure
    from unittest.mock import patch
    with patch("dispatcher.main.validate_jwt", return_value={"user": "test_user", "email": "test@test.com"}):
        response = client.post(
            "/v1/chat/completions",
            json=request_payload,
            headers={"Authorization": "Bearer test_token"},
        )
        print(">> API RESPONSE: ", response)
        message = response.json()["choices"][0]["message"]["content"]
        print(">> LLM RSESPONSE", message)

        qa["response"] = message

        

In [ ]:
# Save the responses to a file

qa_output_file = os.path.join(data_path, "responses", qa_source_file)

print(f"Saving responses to {qa_output_file}")
with open(qa_output_file, 'w') as f:
    f.write(json.dumps(qa_data, indent=2))